In [1]:
!pip install langchain langchain-community langchain-core

  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.5.0
    Uninstalling typing-extensions-4.5.0:
      Successfully uninstalled typing-extensions-4.5.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic-core 2.16.2
    Uninstalling pydantic-core-2.16.2:
      Successfully uninstalled pydantic-core-2.16.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: yarl
    Found existing installation: yarl 1.6.3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-baselines3 2.3.2 requires gymnasium<0.30,>=0.28.1, but you have gymnasium 1.0.0a2 which is incompatible.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.
distributed 2022.2.1 requires cloudpickle>=1.5.0, but you have cloudpickle 1.2.2 which is incompatible.


In [16]:
!pip install pypdf

In [17]:
!pip install -qU "langchain-chroma>=0.1.2"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.29.2 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 5.29.2 which is incompatible.
stable-baselines3 2.3.2 requires gymnasium<0.30,>=0.28.1, but you have gymnasium 1.0.0a2 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.


In [19]:
!pip install protobuf==3.20.*

  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.2
    Uninstalling protobuf-5.29.2:
      Successfully uninstalled protobuf-5.29.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\andre\\anaconda3\\Lib\\site-packages\\google\\~upb\\_message.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
#from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
# from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pypdf

In [2]:
model_local = ChatOllama(model="llama3.2")

C:\Users\andre\AppData\Local\Temp\ipykernel_2412\2362477809.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model_local = ChatOllama(model="llama3.2")


In [3]:
# loader = PyPDFLoader("../../datasets/medEncyclopedia.pdf")
# docs = loader.load()

#read doc
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents
docs = read_doc("../../datasets/book")

In [4]:
#Text Splitting
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=50,
)
doc_splits = splitter.split_documents(docs)

In [5]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="medassist-chroma",
    embedding_function=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
    persist_directory="./chroma_db",
)

# vectorstore = Chroma.from_documents(
#         documents=doc_splits,
#         collection_name="medassist-chroma",
#         embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
#     )


C:\Users\andre\AppData\Local\Temp\ipykernel_2412\3364344991.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_function=embeddings.OllamaEmbeddings(model='nomic-embed-text'),


In [7]:
retriever = vector_store.as_retriever()

In [8]:
# print("Before RAG\n")
# before_rag_template = "What is {topic}"
# before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
# before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
# print(before_rag_chain.invoke({"topic": "effusion"}))

Before RAG

Effusion is a process in which a gas slowly escapes from a container or a small opening, often due to pressure differences between the inside and outside of the container. This phenomenon occurs when the gas molecules have enough energy to overcome the surface tension of the liquid or solid boundary surrounding them.

There are several types of effusions:

1. **Diffusion**: A slow and random movement of particles from an area of higher concentration to an area of lower concentration, resulting in uniform distribution.
2. **Evaporation**: The process by which a liquid transforms into a gas, often accompanied by the escape of gas molecules through a small opening or pore.
3. **Siphoning**: A type of effusion where a liquid is drawn out of a container by capillary action and pressure differences.

Effusion has various applications in:

1. **Scientific research**: Studying the properties of gases, such as molecular weights and viscosities.
2. **Materials science**: Analyzing th

In [11]:
print("\n########\nAfter RAG\n")
after_rag_template = """You are a chatbot answering medical questions related to chest diseases. Do not provide any medical consultations. Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("What is edema?"))



########
After RAG

Edema refers to the accumulation of excess fluid in the interstitial tissue, which can cause swelling in various parts of the body, including the chest. In the context of chest diseases, edema can occur due to increased fluid retention, often as a result of heart failure or lung disease, such as pulmonary congestion or pleural effusion.
